*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [22]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, Flatten, Dense

print(tf.__version__)

2.12.0-dev20221212


In [73]:
train_file_path = "data/train-data.tsv"
test_file_path = "data/valid-data.tsv"
header = ["labels", "text"]

train_df = pd.read_csv(train_file_path, sep="\t", header = None, names = header)
test_df = pd.read_csv(test_file_path, sep="\t", header = None, names = header)

train_text, test_text = train_df.text, test_df.text
train_labels = np.where(train_df.labels == "ham", 0, 1)
test_labels = np.where(test_df.labels == "ham", 0, 1)


In [84]:
def text_encoded(tokenizer):
    def inner(text, maxlen, truncating = "post", padding = "post"):
        sequences = tokenizer.texts_to_sequences(text)
        padded = pad_sequences(sequences, maxlen = maxlen, padding = padding, truncating = truncating)
        return padded
    return inner



In [85]:
# following this this preprocessing and modeling tutorial 
# https://towardsdatascience.com/nlp-detecting-spam-messages-with-tensorflow-b12195b8cf0e

vocab_size = 1000
embedding_dim = 16
oov_tok = "<OOV>"
max_length = 100

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index
text_to_sequences = text_encoded(tokenizer)

In [104]:
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Flatten(),
    Dense(30, activation='relu'),
    Dense(6, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 16)           16000     
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_10 (Dense)            (None, 30)                48030     
                                                                 
 dense_11 (Dense)            (None, 6)                 186       
                                                                 
 dense_12 (Dense)            (None, 1)                 7         
                                                                 
Total params: 64,223
Trainable params: 64,223
Non-trainable params: 0
_________________________________________________________________


In [113]:
num_epochs = 200
history = model.fit(padded, train_labels, epochs = num_epochs, validation_data = (test_padded, test_labels))

Epoch 1/200
131/131 [==============================] - 0s 3ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 0.0553 - val_accuracy: 0.9878
Epoch 2/200
131/131 [==============================] - 0s 3ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0458 - val_accuracy: 0.9885
Epoch 3/200
131/131 [==============================] - 1s 6ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0592 - val_accuracy: 0.9885
Epoch 4/200
131/131 [==============================] - 1s 6ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0625 - val_accuracy: 0.9878
Epoch 5/200
131/131 [==============================] - 1s 5ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 0.0598 - val_accuracy: 0.9878
Epoch 6/200
131/131 [==============================] - 1s 5ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.0524 - val_accuracy: 0.9885
Epoch 7/200
131/131 [==============================] - 1s 5ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0578 - val_accuracy: 0.9878

In [114]:
# add loss plot here


In [115]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text):
    features = text_to_sequences([text], max_length)
    prediction = float(model.predict(features))
    return prediction, "ham" if prediction < 0.5 else "spam"


pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 40ms/step
(4.493818858697374e-11, 'ham')


In [116]:
predict_message("sale today! to stop texts call 98912460324")

1/1 [==============================] - 0s 26ms/step


(0.000114406444481574, 'ham')

In [117]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 22ms/step
You haven't passed yet. Keep trying.


# Run Once

Get the data into a local storage folder

Requirement: make a `data` folder here.

In [2]:
import requests
import os

urls = ["https://cdn.freecodecamp.org/project-data/sms/train-data.tsv", "https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv"]

for url in urls:
    filename = os.path.join("data", os.path.basename(url))
    with open(filename , 'wb') as fh:
        fh.write(requests.get(url).content)